In [37]:
import sys
import nltk
import sklearn
import pandas as pd
import numpy as np

# Load DataSet

In [38]:
df = pd.read_json('C:/Users/admin/Desktop/SIH/Sentiment Analysis/Musical_Instruments.json',lines=True)

In [39]:
df.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,1384719342,"[0, 0]",5,"Not much to write about here, but it does exac...","02 28, 2014",A2IBPI20UZIR0U,"cassandra tu ""Yeah, well, that's just like, u...",good,1393545600
1,1384719342,"[13, 14]",5,The product does exactly as it should and is q...,"03 16, 2013",A14VAT5EAX3D9S,Jake,Jake,1363392000
2,1384719342,"[1, 1]",5,The primary job of this device is to block the...,"08 28, 2013",A195EZSQDW3E21,"Rick Bennette ""Rick Bennette""",It Does The Job Well,1377648000
3,1384719342,"[0, 0]",5,Nice windscreen protects my MXL mic and preven...,"02 14, 2014",A2C00NNG1ZQQG2,"RustyBill ""Sunday Rocker""",GOOD WINDSCREEN FOR THE MONEY,1392336000
4,1384719342,"[0, 0]",5,This pop filter is great. It looks and perform...,"02 21, 2014",A94QU4C90B1AX,SEAN MASLANKA,No more pops when I record my vocals.,1392940800


In [40]:
df = df.drop(labels = ['asin'],axis=1)
df = df.drop(labels = ['helpful'],axis=1)
df = df.drop(labels = ['unixReviewTime'],axis=1)
df = df.drop(labels = ['reviewerName'],axis=1)
df = df.drop(labels = ['reviewerID'],axis=1)
df = df.drop(labels = ['reviewTime'],axis=1)
df = df.drop(labels = ['summary'],axis=1)
df.head()

,overall,reviewText
0,5,"Not much to write about here, but it does exac..."
1,5,The product does exactly as it should and is q...
2,5,The primary job of this device is to block the...
3,5,Nice windscreen protects my MXL mic and preven...
4,5,This pop filter is great. It looks and perform...


In [41]:
#useful infromation about the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10261 entries, 0 to 10260
Data columns (total 2 columns):
overall       10261 non-null int64
reviewText    10261 non-null object
dtypes: int64(1), object(1)
memory usage: 160.4+ KB


In [42]:
ratings = df['overall']
print(ratings.value_counts())

5    6938
4    2084
3     772
2     250
1     217
Name: overall, dtype: int64


# Preprocessing

In [43]:
feedback = df['reviewText']
feedback[:10]

0    Not much to write about here, but it does exac...
1    The product does exactly as it should and is q...
2    The primary job of this device is to block the...
3    Nice windscreen protects my MXL mic and preven...
4    This pop filter is great. It looks and perform...
5    So good that I bought another one.  Love the h...
6    I have used monster cables for years, and with...
7    I now use this cable to run from the output of...
8    Perfect for my Epiphone Sheraton II.  Monster ...
9    Monster makes the best cables and a lifetime w...
Name: reviewText, dtype: object

In [44]:
processed = feedback.str.replace(r'[0-9]', '')
processed = processed.str.replace(r'_', '')
processed = processed.str.lower()
processed

0        not much to write about here, but it does exac...
1        the product does exactly as it should and is q...
2        the primary job of this device is to block the...
3        nice windscreen protects my mxl mic and preven...
4        this pop filter is great. it looks and perform...
5        so good that i bought another one.  love the h...
6        i have used monster cables for years, and with...
7        i now use this cable to run from the output of...
8        perfect for my epiphone sheraton ii.  monster ...
9        monster makes the best cables and a lifetime w...
10       monster makes a wide array of cables, includin...
11       i got it to have it if i needed it. i have fou...
12       if you are not use to using a large sustaining...
13       i love it, i used this for my yamaha ypt- and ...
14       i bought this to use in my home studio to cont...
15       i bought this to use with my keyboard. i wasn'...
16       this fender cable is the perfect length for me.

In [46]:
# remove punctuations

import string

def remove_punctuation(txt):
    txt_nopunct = "".join([c for c in txt if c not in string.punctuation])
    return txt_nopunct

processed = processed.apply(lambda x:remove_punctuation(x))

In [47]:
# remove stop words

from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

processed = processed.apply(lambda x : ' '.join(term for term in x.split() if term not in stop_words))

In [51]:
processed[:10]

0    much write exactli suppos filter pop sound rec...
1    product exactli quit affordablei realiz doubl ...
2    primari job devic block breath would otherwis ...
3    nice windscreen protect mxl mic prevent pop th...
4    pop filter great look perform like studio filt...
5    good bought anoth one love heavi cord gold con...
6    use monster cabl year good reason lifetim warr...
7    use cabl run output pedal chain input fender a...
8    perfect epiphon sheraton ii monster cabl well ...
9    monster make best cabl lifetim warranti doesnt...
Name: reviewText, dtype: object

In [52]:
# stemming

ps = nltk.PorterStemmer()

processed = processed.apply(lambda x : ' '.join(ps.stem(term) for term in x.split()))

In [53]:
processed[:18]

0     much write exactli suppo filter pop sound reco...
1     product exactli quit affordablei realiz doubl ...
2     primari job devic block breath would otherwi p...
3     nice windscreen protect mxl mic prevent pop th...
4     pop filter great look perform like studio filt...
5     good bought anoth one love heavi cord gold con...
6     use monster cabl year good reason lifetim warr...
7     use cabl run output pedal chain input fender a...
8     perfect epiphon sheraton ii monster cabl well ...
9     monster make best cabl lifetim warranti doesnt...
10    monster make wide array cabl includ high end i...
11    got need found dont realli need often rare use...
12    use use larg sustain pedal play piano may appe...
13    love use yamaha ypt work great would recommend...
14    bought use home studio control midi keyboard want
15    bought use keyboard wasnt realli awar option k...
16    fender cabl perfect length sometim find bit lo...
17    want look aloneit nice look cord know perf

In [54]:
#tokenizing

from nltk.tokenize import word_tokenize
all_words = []

for feedback in processed:
    words = word_tokenize(feedback)
    for w in words:
        all_words.append(w)
        
all_words = nltk.FreqDist(all_words)

In [59]:
#all_words

print(len(all_words))

#print(all_words.most_common(25))

22196


In [60]:
#words as features

#print(all_words.keys())

word_features  = list(all_words.keys())[:5000]
word_features

['much',
 'write',
 'exactli',
 'suppo',
 'filter',
 'pop',
 'sound',
 'record',
 'crisp',
 'one',
 'lowest',
 'price',
 'amazon',
 'might',
 'well',
 'buy',
 'honestli',
 'work',
 'despit',
 'product',
 'quit',
 'affordablei',
 'realiz',
 'doubl',
 'screen',
 'arriv',
 'even',
 'better',
 'expecteda',
 'ad',
 'bonu',
 'carri',
 'small',
 'hint',
 'smell',
 'old',
 'grape',
 'candi',
 'use',
 'reminisc',
 'sake',
 'can',
 'not',
 'stop',
 'put',
 'next',
 'nose',
 'dif',
 'need',
 'expen',
 'may',
 'come',
 'plea',
 'aroma',
 'like',
 'mine',
 'didbuy',
 'primari',
 'job',
 'devic',
 'block',
 'breath',
 'would',
 'otherwi',
 'produc',
 'allow',
 'voic',
 'pass',
 'notic',
 'reduct',
 'volum',
 'high',
 'frequenc',
 'cloth',
 'let',
 'color',
 'metal',
 'clamp',
 'mount',
 'attach',
 'mike',
 'stand',
 'secur',
 'enough',
 'keep',
 'goo',
 'neck',
 'littl',
 'coax',
 'stay',
 'nice',
 'windscreen',
 'protect',
 'mxl',
 'mic',
 'prevent',
 'thing',
 'gooseneck',
 'margin',
 'abl',
 'hol

In [28]:
#find features

def find_features(feedback):
    words = word_tokenize(feedback)
    features = {}
    for word in word_features:
        features[word] = (word in words)
    return features

#example

feature = find_features(processed[0])
for key,value in feature.items():
    if(value==True):
        print(key)

much
write
exactli
suppos
filter
pop
sound
record
crisp
one
lowest
price
amazon
might
well
buy
honestli
work
despit


In [29]:
print(processed[0])
#print(feature)
d = processed.apply(lambda x : ' '.join(term for term in x.split() if term in word_features))
print(d[0])

much write exactli suppos filter pop sound record much crisp one lowest price pop filter amazon might well buy honestli work despit price
much write exactli suppos filter pop sound record much crisp one lowest price pop filter amazon might well buy honestli work despit price


In [30]:
#d
x = processed
y = df['overall']
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(d,y,test_size=0.25,random_state=42)

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(x_train)
vectors = vectorizer.transform(x_train)
#print(vectorizer.get_feature_names())

from sklearn.svm import LinearSVC
clf=LinearSVC()

from sklearn.pipeline import Pipeline
text_clf = Pipeline([('tfidf',TfidfVectorizer()), ('clf',LinearSVC())])

text_clf.fit(x_train,y_train)

predictions = text_clf.predict(x_test)
from sklearn import metrics
from sklearn.metrics import confusion_matrix,classification_report

print(metrics.accuracy_score(y_test,predictions))

predictions = text_clf.predict(x_test)
from sklearn import metrics
print(metrics.classification_report(y_test, predictions))

0.6687451286048324
             precision    recall  f1-score   support

          1       0.56      0.15      0.24        60
          2       0.21      0.05      0.08        60
          3       0.25      0.13      0.17       189
          4       0.37      0.26      0.31       528
          5       0.75      0.89      0.81      1729

avg / total       0.62      0.67      0.63      2566



In [31]:
#find features for all messages


feedback = list(zip(processed,ratings))

seed = 1
np.random.seed = seed
np.random.shuffle(feedback)

featuresets = [(find_features(text),label) for (text,label) in feedback]

# Splitting

In [32]:
from sklearn.model_selection import train_test_split

training, testing = train_test_split(featuresets,test_size=0.33,random_state=42)

In [33]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,classification_report

In [34]:
names = ['Random Forest']
classifiers = [RandomForestClassifier()]
models = list(zip(names, classifiers))


In [35]:
from nltk.classify.scikitlearn import SklearnClassifier

for name, model in models :
    nltk_model = SklearnClassifier(model)
    nltk_model.train(training)
    accuracy = nltk.classify.accuracy(nltk_model, testing) * 100
    print('{}: Accuracy: {}'.format(name, accuracy))

Random Forest: Accuracy: 65.21995866548568
